In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [2]:
import os
os.chdir('../')

In [32]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import tcav
from src.download_data import download_imagenet, download_random_imagenet_classes
import pickle
from src.dataset import *
from src.concept_vectors import *
from src.util import *
import random
from src.plots import *
from sklearn.manifold import TSNE
from scipy.spatial.distance import cosine 
import pandas as pd
from sklearn.datasets import make_moons
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering
from sklearn.datasets import make_blobs
from src.hierarchy import *
from src.metrics import *
import pickle
import tensorflow as tf
import keras
from keras.models import load_model, Model
import tcav.model  as model
import keras
from keras.applications.vgg16 import VGG16
from keras import losses

In [10]:
m = tf.keras.applications.VGG16(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
)


In [11]:
class VggWrapper(model.KerasModelWrapper):
    def get_image_shape(self):
        return np.array([128,128,3])

In [5]:
class ResnetWrapper(model.KerasModelWrapper):
    def get_image_shape(self):
        return np.array([224,224,3])

In [12]:
m.compile(optimizer='Adam',
              loss='sparse_categorical_crossentropy')

In [14]:
a = np.array(m.get_weights())

/auto/homes/njr61/environments/mambaforge/envs/concepts/lib/python3.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


In [20]:
a[1]

array([ 0.73429835,  0.09340367,  0.06775674,  0.8862966 ,  0.25994542,
        0.66426694, -0.01582893,  0.3249065 ,  0.68600726,  0.06247932,
        0.58156496,  0.2361475 ,  0.69694996,  0.19451167,  0.4858922 ,
        0.44571847,  0.5113422 ,  0.208576  ,  0.57557714,  0.33199573,
        0.4997983 ,  0.7117759 ,  0.30284074,  0.7082712 ,  0.04548979,
        0.7446502 ,  0.29845494,  0.48211655,  0.81658626,  0.62603897,
        0.3768093 ,  2.064037  ,  0.77311045,  0.3459577 ,  0.6130958 ,
        0.65459156,  0.39045632,  0.50869167,  0.2625384 ,  0.23669638,
        0.07971057,  1.1179353 ,  0.26129362,  0.8697589 ,  0.21543622,
        0.78007823,  0.37015367,  0.47993386,  0.4313978 ,  0.5084194 ,
        0.23049663,  0.7636527 ,  0.35419866,  0.45794216,  0.4662595 ,
        0.09850298,  0.3803252 ,  0.66880196,  0.4015123 ,  0.90510356,
        0.43166816,  1.302014  ,  0.5306885 ,  0.48993504], dtype=float32)

In [30]:
responsive_weights(m.get_weights())

[array([[[[-3.92414004e-01, -3.52556743e-02,  9.87144373e-03, ...,
           -2.18004733e-02,  3.49400863e-02, -3.93940893e-04],
          [-4.69210111e-02,  6.35051588e-03,  1.69788375e-02, ...,
            3.81944925e-02,  1.59282908e-02,  8.66925064e-03],
          [-3.67878437e-01,  6.89901761e-04,  6.39618887e-03, ...,
            1.47388428e-02,  3.89889590e-02,  7.96111207e-03]],
 
         [[-7.25840777e-02,  5.59377111e-02, -1.06522976e-03, ...,
           -5.36507973e-03, -3.07302568e-02,  3.51585969e-02],
          [-3.58317345e-01,  4.54415344e-02,  1.21036889e-02, ...,
           -6.12554774e-02,  1.37692437e-01,  5.59128746e-02],
          [-2.60391146e-01,  9.31562260e-02, -2.14987108e-03, ...,
           -6.74050301e-02, -8.28193799e-02,  3.06213088e-02]],
 
         [[ 5.06991521e-02,  9.86070111e-02,  1.03369355e-01, ...,
            1.41179943e-02, -3.17566097e-01, -2.76883155e-01],
          [-1.35417108e-03,  1.96915474e-02,  1.24452151e-02, ...,
           -9.407

In [21]:
m.save('./dataset/models/keras/model_vgg16.h5')

In [36]:
save_vgg_model(lambda w: w,'./dataset/models/keras/model_vgg16.h5')

In [34]:
save_vgg_model(perturb_weights,'./dataset/models/keras/model_vgg16_robust.h5')

/auto/homes/njr61/main_code/src/util.py:322: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  weights = np.array(m.get_weights())


In [35]:
save_vgg_model(responsive_weights,'./dataset/models/keras/model_vgg16_responsive.h5')

/auto/homes/njr61/main_code/src/util.py:322: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  weights = np.array(m.get_weights())


In [9]:
sess = utils.create_session()
GRAPH_PATH = "./dataset/models/keras/model_vgg16.h5"
LABEL_PATH = "./dataset/models/inception5h/imagenet_comp_graph_label_strings.txt"
mymodel = ResnetWrapper(sess,
                                GRAPH_PATH,
                                LABEL_PATH)

/auto/homes/njr61/environments/mambaforge/envs/concepts/lib/python3.7/site-packages/tensorflow/python/client/session.py:1769: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Instructions for updating:
Colocations handled automatically by placer.


2023-01-20 09:25:10.370896: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/auto/homes/njr61/environments/mambaforge/envs/concepts/lib/python3.7/site-packages/tensorflow/python/framework/ops.py", line 2693, in get_attr
    pywrap_tf_session.TF_OperationGetAttrValueProto(self._c_op, name, buf)
tensorflow.python.framework.errors_impl.InvalidArgumentError: Operation 'conv4_block3_2_bn/cond' has no attr named '_class'.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/auto/homes/njr61/environments/mambaforge/envs/concepts/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_9307/3980073822.py", line 6, in <module>
    LABEL_PATH)
  File "/auto/homes/njr61/environments/mambaforge/envs/concepts/lib/python3.7/site-packages/tcav/model.py", line 451, in __init__
    self._make_gradient_tensors()
  File "/auto/homes/njr61/environments/mambaforge/envs/conce

TypeError: object of type 'NoneType' has no len()

In [47]:
image_dir = "./dataset/images"
activation_dir = './results/activations'
max_examples = 100
act_generator = act_gen.ImageActivationGenerator(mymodel, image_dir, activation_dir,max_examples=max_examples)

In [48]:
examples = act_generator.get_examples_for_concept("0_number")

In [49]:
len(examples)

100

In [4]:
mnist_attributes = get_mnist_attributes()

In [5]:
robustness_image_metric(create_linkage_hierarchy,load_tcav_vectors_simple,'mnist',mnist_attributes,[43,44,45])

46.666666666666664

In [6]:
responsiveness_image_metric(create_linkage_hierarchy,load_tcav_vectors_simple,'mnist',mnist_attributes,[43,44,45])

46.0

In [9]:
responsiveness_image_metric(create_linkage_hierarchy,
                                       load_label_vectors_simple,'mnist',mnist_attributes,[43,44,45])

47.0

In [11]:
sess = utils.create_session()

2023-01-18 10:35:02.110670: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-18 10:35:02.118093: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [12]:
GRAPH_PATH = "./dataset/models/inception5h/tensorflow_inception_graph.pb"
LABEL_PATH = "./dataset/models/inception5h/imagenet_comp_graph_label_strings.txt"
mymodel = model.GoogleNetWrapper_public(sess,
                                    GRAPH_PATH,
                                    LABEL_PATH)

In [13]:
activation_dir = './results/activations'
working_dir = './results/tmp'
image_dir = "./dataset/images"

In [15]:
act_generator = act_gen.ImageActivationGenerator(mymodel, 
                                                 image_dir, activation_dir, max_examples=100)

In [18]:
d = act_generator.get_examples_for_concept("0_number")

In [19]:
d.shape

(100, 224, 224, 3)

In [21]:
activations = load_activations_tcav(mnist_attributes,experiment_name="mnist",max_examples=100)

2023-01-18 10:40:48.060535: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled


In [25]:
np.linalg.norm(activations["0_number"])

82407.38

In [5]:
hierarchy_method=create_linkage_hierarchy
embedding_method=load_label_vectors_simple
dataset='mnist'
attributes=mnist_attributes
random_seeds=[43,44,45]
model="GoogleNet"

In [6]:
random.seed(42)
    
n_concepts = len(attributes)//5
compare_concepts = len(attributes)//5

selected_concepts = random.sample(attributes,k=n_concepts)

avg_truthfulness = []

In [7]:
seed = 43
concept = "0_number"

In [8]:
activation_dir = './results/activations'
working_dir = './results/tmp'
image_dir = "./dataset/images"

max_images = 100

if dataset == 'mnist':
    concepts = get_mnist_attributes()

In [9]:
activations = load_activations_tcav(concepts,experiment_name=dataset,max_examples=max_images)

2023-01-18 11:31:35.994492: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-18 11:31:36.002495: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2023-01-18 11:31:40.246849: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled


In [10]:
our_vectors = activations[concept]

In [11]:
cdist

<function scipy.spatial.distance.cdist(XA, XB, metric='euclidean', *, out=None, **kwargs)>

In [13]:
all_distance_pairs = []
for other_concept in concepts:
    print(other_concept)
    if other_concept != concept:
        distances = cdist(our_vectors,activations[other_concept],metric='cosine')
        all_distance_pairs.append((other_concept,np.mean(distances)))

0_color
0_number
1_color
1_number
2_color
2_number
3_color
3_number
4_color
4_number
5_color
5_number
6_color
6_number
7_color
7_number
8_color
8_number
9_color
9_number
spurious


In [14]:
all_distance_pairs = sorted(all_distance_pairs,key=lambda k: k[1])
[i[0] for i in all_distance_pairs][:5]

['0_color', '5_number', '6_number', '5_color', '6_color']

In [15]:
all_distance_pairs

[('0_color', 0.2703246392913351),
 ('5_number', 0.3634966880977542),
 ('6_number', 0.36597444109862576),
 ('5_color', 0.36602611221781145),
 ('6_color', 0.3692161675511568),
 ('9_number', 0.37112409359930537),
 ('8_number', 0.3792494928402702),
 ('9_color', 0.3808778023167217),
 ('8_color', 0.3813513889149904),
 ('3_number', 0.38222491701495953),
 ('spurious', 0.3834654850094098),
 ('3_color', 0.39138542483929173),
 ('2_number', 0.40489694623799805),
 ('2_color', 0.4084804230132554),
 ('4_number', 0.42284139339690574),
 ('1_color', 0.42325033832208264),
 ('7_color', 0.42465486258556595),
 ('4_color', 0.42499817839442877),
 ('1_number', 0.4272763922677328),
 ('7_number', 0.4313626597133339)]

In [16]:
co_occuring_concepts = find_similar_concepts(concept,dataset,model,compare_concepts,seed)

/auto/homes/njr61/environments/mambaforge/envs/concepts/lib/python3.7/site-packages/tensorflow/python/client/session.py:1769: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [19]:
co_occuring_concepts_hierarchy = rank_distance_concepts(embedding_method,dataset,concept,
                                                                    co_occuring_concepts,seed)

In [22]:
co_occuring_concepts

['0_color', '6_color', '5_number', '5_color']

In [21]:
co_occuring_concepts_hierarchy

['0_color', '6_color', '5_number', '5_color']

In [26]:
stats.kendalltau(co_occuring_concepts,co_occuring_concepts_hierarchy).correlation

1.0

In [5]:
truthfulness_metric(create_linkage_hierarchy,load_tcav_vectors_simple,
                    'mnist',mnist_attributes,[43,44])

2023-01-18 12:07:52.800447: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-18 12:07:52.807738: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2023-01-18 12:07:56.934859: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
/auto/homes/njr61/environments/mambaforge/envs/concepts/lib/python3.7/site-packages/tensorflow/python/client/session.py:1769: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources he

0.5833333333333334

In [9]:
import gc; gc.collect()

21

In [10]:
truthfulness_metric(create_linkage_hierarchy,load_label_vectors_simple,
                    'mnist',mnist_attributes,[43,44])

RuntimeError: can't start new thread

In [38]:
hierarchy_method = create_linkage_hierarchy
embedding_method = load_tcav_vectors_simple
dataset = 'mnist'
attributes = mnist_attributes[:4]
random_seeds = [43,44,45]
suffix = '_image_responsiveness'

In [42]:
baseline_hierarchies = [create_hierarchy(hierarchy_method,embedding_method,dataset,attributes,seed) for seed in random_seeds]
robustness_hierarchies = [create_hierarchy(hierarchy_method,embedding_method,dataset+'_image_robustness',attributes,seed) for seed in random_seeds]
responsiveness_hierarchies = [create_hierarchy(hierarchy_method,embedding_method,dataset+'_image_responsiveness',attributes,seed) for seed in random_seeds]

In [43]:
responsiveness_hierarchies[0].distance(baseline_hierarchies[0])

4.0

In [44]:
robustness_hierarchies[0].distance(baseline_hierarchies[0])

4.0